In [1]:
import kfp
from kfp import dsl
import pickle
import os
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [2]:
@dsl.component(base_image="quay.io/jupyter/scipy-notebook:lab-4.4.3")
def preprocess_op(train_path: dsl.Output[dsl.Artifact],
                  test_path: dsl.Output[dsl.Artifact]):
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    import pickle

    X, y = load_iris(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    with open(train_path.path, 'wb') as f:
        pickle.dump((X_train, y_train), f)
    with open(test_path.path, 'wb') as f:
        pickle.dump((X_test, y_test), f)


In [3]:
@dsl.component(base_image="quay.io/jupyter/scipy-notebook:lab-4.4.3")
def train_op(train_data: dsl.Input[dsl.Artifact],
             model_output: dsl.Output[dsl.Model]):
    import pickle
    from sklearn.ensemble import RandomForestClassifier

    with open(train_data.path, 'rb') as f:
        X_train, y_train = pickle.load(f)

    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    with open(model_output.path, 'wb') as f:
        pickle.dump(clf, f)

In [4]:
@dsl.component(base_image="quay.io/jupyter/scipy-notebook:lab-4.4.3")
def eval_op(test_data: dsl.Input[dsl.Artifact],
            model_input: dsl.Input[dsl.Model]):
    import pickle
    from sklearn.metrics import accuracy_score

    with open(test_data.path, 'rb') as f:
        X_test, y_test = pickle.load(f)
    with open(model_input.path, 'rb') as f:
        clf = pickle.load(f)

    acc = accuracy_score(y_test, clf.predict(X_test))
    print(f"Model accuracy: {acc}")


In [5]:
@dsl.pipeline(name="iris-classification-v2")
def iris_pipeline():
    preprocess_task = preprocess_op()
    train_task = train_op(train_data=preprocess_task.outputs["train_path"])
    eval_task = eval_op(
        test_data=preprocess_task.outputs["test_path"],
        model_input=train_task.outputs["model_output"]
    )


In [6]:
from kfp import compiler
compiler.Compiler().compile(
    pipeline_func=iris_pipeline,
    package_path="iris_pipeline_v4.yaml"
)